In [15]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

# Define camera parameters for left and right cameras
left_camera_intrinsic = np.array([[1000.0, 0.0, 360.0],
                                  [0.0, 1000.0, 640.0],
                                  [0.0, 0.0, 1.0]])

left_camera_extrinsic = np.array([[0.88649035, -0.46274707, -0.00, -14.42428],
                                  [-0.070794605, -0.13562201, -0.98822814, 86.532959],
                                  [0.45729965, 0.8760547, -0.1529876, 235.35446]])

right_camera_intrinsic = np.array([[1100.0, 0.0, 360.0],
                                   [0.0, 1100.0, 640.0],
                                   [0.0, 0.0, 1.0]])

right_camera_extrinsic = np.array([[0.98480779, -0.17364818, -4.9342116E-8, -0.98420829],
                                   [-0.026566068, -0.15066338, -0.98822814, 85.070221],
                                   [0.17160401, 0.97321475, -0.1529876, 236.97873]])

fundamental_matrix = np.array([[3.283965767647195E-7, -6.76098398189792E-6, 0.0021123144539793737],
                               [-8.046341661808292E-6, 3.05632173594769E-8, 0.05124913199417346],
                               [0.0048160232373805345, -0.051062699158041805, 1.0706286680443888]])


# Palette Color
palette_file = 'SBS images/000.jpg'
palette = cv2.imread(palette_file)
palette = cv2.cvtColor(palette, cv2.COLOR_BGR2RGB)



def split_images(image):
    # Split the image into left and right images
    height, width, _ = image.shape
    left_img = image[:, :width // 2]
    right_img = image[:, width // 2:]
    return left_img, right_img

def find_brightest_blue_pixels(image):
    # Create a mask for the region of gnome
    mask = np.zeros(image.shape[:2], np.uint8)
    mask[0:875, 160:600] = 255
    image = cv2.bitwise_and(src1=image, src2=image, mask=mask)

    brightest_pixels = []
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define blue color ranges
    lower_blue = np.array([100, 70, 50])
    upper_blue = np.array([150, 255, 255])
    mask1 = cv2.inRange(hsv, lower_blue, upper_blue)
    
    lower_dark_blue = np.array([100, 50, 100])
    upper_dark_blue = np.array([120, 255, 255])
    mask2 = cv2.inRange(hsv, lower_dark_blue, upper_dark_blue)
    
    combined_mask = cv2.bitwise_or(mask1, mask2)
    combined_mask = cv2.bitwise_and(src1=image, src2=image, mask=combined_mask)

    #plt.imshow(combined_mask)
    for row in range(combined_mask.shape[0]):
        pixels = combined_mask[row, :, 2]
        if np.any(pixels):
            brightest_pixel = np.argmax(pixels)
            brightest_pixels.append((brightest_pixel, row))
            image[row, brightest_pixel] = (0, 255, 0)

    return brightest_pixels

def compute_epipolar_lines(points_left):
    # Compute epipolar lines using the fundamental matrix
    points_left = np.array(points_left)
    num_points = points_left.shape[0]
    ones = np.ones((num_points, 1))
    points_left_homogeneous = np.hstack((points_left, ones))
    lines_left = (fundamental_matrix @ points_left_homogeneous.T).T
    
    
    return lines_left

def find_corresponding_point_in_line(line, right_points):
    # Find most corresponding points in a line
    points_list = []


    for line_coeffs in line:
        a, b, c = line_coeffs
        if b == 0: 
            points_list.append(None)
            continue

        min = float('inf')
        corresponding_point = None

        for rp in right_points:
            x, y = rp[:2]
            err = abs(a * x + b * y + c)

            if err < min:
                min = err
                corresponding_point = rp

        points_list.append(corresponding_point)

    return points_list



def triangulate_points(f_points, epipolar_points, left_intr, left_ext, right_intr, right_ext):
    # Triangulate 3D points algorithm from corresponding 2D points
    points_3d = []
    colors = []

    for f_pt, e_pt in zip(f_points, epipolar_points):
        if e_pt is None:
            continue

        x_left, y_left = f_pt
        x_right, y_right = e_pt
        
        p_left = np.array([x_left, y_left, 1.0])
        p_right = np.array([x_right, y_right, 1.0])

        P1 = left_intr @ left_ext
        P2 = right_intr @ right_ext

        A = np.vstack(((p_left[0] * P1[2]) - P1[0],
                       (p_left[1] * P1[2]) - P1[1],
                       (p_right[0] * P2[2]) - P2[0],
                       (p_right[1] * P2[2]) - P2[1]))

        _, _, vh = np.linalg.svd(A)
        point_3d_hom = vh[3] / vh[3, 3]
        e = A @ point_3d_hom
        error = np.linalg.norm(e)

        if error <= 20:
            points_3d.append(point_3d_hom[:3])
            colors.append((x_left, y_left))

        

    colors = coloring(palette, colors)

    return np.array(points_3d), colors

def coloring(image, points):
    # Get colors from image based on points
    colors = []
    for point in points:
        if point is not None:
            x, y = map(int, point)
            if 0 <= y < image.shape[0] and 0 <= x < image.shape[1]:
                color = image[y, x]
                colors.append(color)
            else:
                colors.append([0, 0, 0])
        else:
            colors.append([0, 0, 0])
    return colors

def save(points_3d,  colors):
    # Save 3D points to a text file
    with open('F11115108.txt', 'w') as f:
        for point_3d, color in zip(points_3d, colors):
            
            x, y, z = point_3d
            r, g, b = color

            # Dirty cleaning of uncorresponding points
            if(abs(x) < 40 or abs(y) < 40 or abs(z) < 40):
                f.write(f"{x} {y} {z} {r} {g} {b}\n")
    # Save 3D points to a text file
    with open('F11115108.xyz', 'w') as f:
        for point_3d, color in zip(points_3d, colors):
            
            x, y, z = point_3d
            r, g, b = color

            # Dirty cleaning of uncorresponding points
            if(abs(x) < 40 or abs(y) < 40 or abs(z) < 40):
                f.write(f"{x} {y} {z} {r} {g} {b}\n")

# Main process
all_points_3d = []
all_corresponding_points = []
all_colors = []



image_files = sorted(glob.glob('SBS images/*.jpg'))

for image_file in image_files:
    image = cv2.imread(image_file)
    left_image, right_image = split_images(image)

    # Pixels
    left_brightest_pixels = find_brightest_blue_pixels(left_image)
    right_brightest_pixels = find_brightest_blue_pixels(right_image)

    # Epipolar line
    epipolar_lines = compute_epipolar_lines(left_brightest_pixels)
    
    # Epipolar points
    epipolar_points = find_corresponding_point_in_line(epipolar_lines, right_brightest_pixels)
    
    # 3D generated model points
    points_3d, colors = triangulate_points(left_brightest_pixels, epipolar_points, left_camera_intrinsic, left_camera_extrinsic, right_camera_intrinsic, right_camera_extrinsic)
    
    all_points_3d.extend(points_3d)
    all_corresponding_points.extend(epipolar_points)
    all_colors.extend(colors)

# Save points
save(all_points_3d, all_colors)
